In [251]:
import pickle
import gzip
import random
import numpy as np

In [252]:

def load_data():
    f = gzip.open(r'C:\Users\Kul Garima\.jupyter\DeepLearningPython35-master\mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)

In [280]:
def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    
    return (training_data, validation_data, test_data)

In [281]:
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    
    return e

In [290]:
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in list(zip(sizes[:-1], sizes[1:]))]
        
    def feedforward(self, a):
        for b, w in list(zip(self.biases, self.weights)):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self,  training_data, epochs, mini_batch_size, eta, test_data = None):
    
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
        n = len(training_data)
        
        for j in range (epochs):
            random.shuffle(training_data)
            mini_batches =[ training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
        
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2} ".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))
    
    def update_mini_batch(self, mini_batch, eta):
        
        bDash = [np.zeros(b.shape) for b in self.biases]
        wDash = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            
            delta_bDash, delta_wDash = self.backprop(x, y)
            
            bDash = [nb+dnb for nb, dnb in list(zip(bDash, delta_bDash))]
            wDash = [nw+dnw for nw, dnw in list(zip(wDash, delta_wDash))]
            
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in list(zip(self.weights, wDash))]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in list(zip(self.biases, bDash))]
        
        
    def backprop(self, x, y):
        
        bDash = [np.zeros(b.shape) for b in self.biases]
        wDash = [np.zeros(w.shape) for w in self.weights]
        
       
        activation = x
        activations = [x] 
        zs = [] 
        for b, w in list(zip(self.biases, self.weights)):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
       
        delta = self.cost_derivative(activations[-1], y)*sigmoid_prime(zs[-1])
        bDash[-1] = delta
        wDash[-1] = np.dot(delta, activations[-2].transpose())
    
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            bDash[-l] = delta
            wDash[-l] = np.dot(delta, activations[-l-1].transpose())
        return (bDash, wDash)
    
    
    def evaluate(self,test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y)


In [291]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))


In [292]:
training_data, validation_data, test_data = load_data_wrapper()


In [293]:

net = Network([784, 30, 10])

In [294]:
net.SGD(training_data, 30, 10, 3.0, test_data = test_data)

Epoch 0: 9050 / 10000 
Epoch 1: 9197 / 10000 
Epoch 2: 9286 / 10000 
Epoch 3: 9360 / 10000 
Epoch 4: 9374 / 10000 
Epoch 5: 9355 / 10000 
Epoch 6: 9433 / 10000 
Epoch 7: 9429 / 10000 
Epoch 8: 9439 / 10000 
Epoch 9: 9445 / 10000 
Epoch 10: 9462 / 10000 
Epoch 11: 9471 / 10000 
Epoch 12: 9476 / 10000 
Epoch 13: 9456 / 10000 
Epoch 14: 9486 / 10000 
Epoch 15: 9471 / 10000 
Epoch 16: 9479 / 10000 
Epoch 17: 9504 / 10000 
Epoch 18: 9497 / 10000 
Epoch 19: 9490 / 10000 
Epoch 20: 9506 / 10000 
Epoch 21: 9497 / 10000 
Epoch 22: 9517 / 10000 
Epoch 23: 9497 / 10000 
Epoch 24: 9536 / 10000 
Epoch 25: 9504 / 10000 
Epoch 26: 9527 / 10000 
Epoch 27: 9513 / 10000 
Epoch 28: 9505 / 10000 
Epoch 29: 9499 / 10000 


In [295]:
net = Network([784, 100, 10])

In [298]:
net.SGD(training_data, 30, 10, 100.0, test_data = test_data)

Epoch 0: 1020 / 10000 
Epoch 1: 1020 / 10000 
Epoch 2: 1020 / 10000 
Epoch 3: 1020 / 10000 
Epoch 4: 1020 / 10000 
Epoch 5: 1019 / 10000 
Epoch 6: 1019 / 10000 
Epoch 7: 1020 / 10000 
Epoch 8: 1020 / 10000 
Epoch 9: 1020 / 10000 
Epoch 10: 1021 / 10000 
Epoch 11: 1021 / 10000 
Epoch 12: 1021 / 10000 
Epoch 13: 1022 / 10000 
Epoch 14: 1022 / 10000 
Epoch 15: 1022 / 10000 
Epoch 16: 1022 / 10000 
Epoch 17: 1022 / 10000 
Epoch 18: 1022 / 10000 
Epoch 19: 1022 / 10000 
Epoch 20: 1022 / 10000 
Epoch 21: 1022 / 10000 
Epoch 22: 1022 / 10000 
Epoch 23: 1022 / 10000 
Epoch 24: 1022 / 10000 
Epoch 25: 1022 / 10000 
Epoch 26: 1022 / 10000 
Epoch 27: 1021 / 10000 
Epoch 28: 1021 / 10000 
Epoch 29: 1021 / 10000 


In [299]:
net = Network([784, 0 , 10])

In [300]:
net.SGD(training_data, 30, 10, 100.0, test_data = test_data)

Epoch 0: 980 / 10000 
Epoch 1: 892 / 10000 
Epoch 2: 958 / 10000 
Epoch 3: 1135 / 10000 
Epoch 4: 980 / 10000 
Epoch 5: 1010 / 10000 
Epoch 6: 1010 / 10000 
Epoch 7: 974 / 10000 
Epoch 8: 974 / 10000 
Epoch 9: 974 / 10000 
Epoch 10: 1032 / 10000 
Epoch 11: 974 / 10000 
Epoch 12: 982 / 10000 
Epoch 13: 982 / 10000 
Epoch 14: 982 / 10000 
Epoch 15: 958 / 10000 
Epoch 16: 958 / 10000 
Epoch 17: 958 / 10000 
Epoch 18: 1009 / 10000 
Epoch 19: 1009 / 10000 
Epoch 20: 1032 / 10000 
Epoch 21: 1009 / 10000 
Epoch 22: 1009 / 10000 
Epoch 23: 974 / 10000 
Epoch 24: 1009 / 10000 
Epoch 25: 1010 / 10000 
Epoch 26: 974 / 10000 
Epoch 27: 974 / 10000 
Epoch 28: 980 / 10000 
Epoch 29: 958 / 10000 
